In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [20]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [21]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 399us/step - 

1052/1052 [==============================] - 0s 88us/step - loss: 22.2979 - val_loss: 25.5585
Epoch 66/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.8945 - val_loss: 25.2575
Epoch 67/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.7306 - val_loss: 26.1722
Epoch 68/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.2052 - val_loss: 26.3090
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.8922 - val_loss: 27.2435
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1388 - val_loss: 25.4379
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.8486 - val_loss: 26.7440
Epoch 72/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.7242 - val_loss: 25.7152
Epoch 73/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.6081 - val_loss: 27.0841
Epoch 74/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 92us/step - loss: 21.7215 - val_loss: 27.6665
Epoch 141/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.8219 - val_loss: 26.2032
Epoch 142/1000
1052/1052 [==============================] - 0s 86us/step - loss: 22.5337 - val_loss: 25.4179
Epoch 143/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.2372 - val_loss: 26.0833
Epoch 144/1000
1052/1052 [==============================] - 0s 85us/step - loss: 22.1166 - val_loss: 26.9362
Epoch 145/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.1603 - val_loss: 25.9194
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 22.1642 - val_loss: 25.6516
Epoch 147/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.3533 - val_loss: 25.7456
Epoch 148/1000
1052/1052 [==============================] - 0s 84us/step - loss: 22.2180 - val_loss: 25.8299
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 92us/step - loss: 17.9158 - val_loss: 21.4375
Epoch 216/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.1118 - val_loss: 23.3570
Epoch 217/1000
1052/1052 [==============================] - 0s 85us/step - loss: 18.7740 - val_loss: 20.7859
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.7075 - val_loss: 20.6965
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.7020 - val_loss: 20.3897
Epoch 220/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.3173 - val_loss: 21.2150
Epoch 221/1000
1052/1052 [==============================] - 0s 89us/step - loss: 17.9452 - val_loss: 20.0531
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.7612 - val_loss: 19.5457
Epoch 223/1000
1052/1052 [==============================] - 0s 87us/step - loss: 17.4764 - val_loss: 19.6622
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 90us/step - loss: 13.3422 - val_loss: 15.6404
Epoch 291/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.5180 - val_loss: 16.3412
Epoch 292/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.1508 - val_loss: 15.6655
Epoch 293/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.4695 - val_loss: 15.4911
Epoch 294/1000
1052/1052 [==============================] - 0s 91us/step - loss: 12.4103 - val_loss: 15.5730
Epoch 295/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.2684 - val_loss: 15.1549
Epoch 296/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.0394 - val_loss: 15.1261
Epoch 297/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.9688 - val_loss: 15.0770
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.6283 - val_loss: 14.2340
Epoch 299/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 87us/step - loss: 8.3117 - val_loss: 10.2706
Epoch 745/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1125 - val_loss: 11.4052
Epoch 746/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2385 - val_loss: 10.5340
Epoch 747/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1222 - val_loss: 9.1133
Epoch 748/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4286 - val_loss: 9.9436
Epoch 749/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2598 - val_loss: 10.3135
Epoch 750/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2456 - val_loss: 10.2018
Epoch 751/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7147 - val_loss: 10.7204
Epoch 752/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2543 - val_loss: 9.7587
Epoch 753/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 86us/step - loss: 8.2556 - val_loss: 9.1751
Epoch 821/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1043 - val_loss: 9.4197
Epoch 822/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.0241 - val_loss: 9.2006
Epoch 823/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6862 - val_loss: 9.5391
Epoch 824/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2006 - val_loss: 10.5668
Epoch 825/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2632 - val_loss: 9.7448
Epoch 826/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7026 - val_loss: 10.2355
Epoch 827/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4549 - val_loss: 9.7356
Epoch 828/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0672 - val_loss: 9.2161
Epoch 829/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 91us/step - loss: 8.1497 - val_loss: 9.2474
Epoch 897/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1175 - val_loss: 9.3009
Epoch 898/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3073 - val_loss: 9.6692
Epoch 899/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3080 - val_loss: 9.4801
Epoch 900/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.0473 - val_loss: 9.6460
Epoch 901/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.5177 - val_loss: 9.6700
Epoch 902/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2902 - val_loss: 9.4714
Epoch 903/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.0448 - val_loss: 12.4588
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.7683 - val_loss: 9.1139
Epoch 905/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 84us/step - loss: 8.3525 - val_loss: 10.5162
Epoch 973/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1708 - val_loss: 11.0281
Epoch 974/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3466 - val_loss: 9.3533
Epoch 975/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2610 - val_loss: 10.0276
Epoch 976/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2585 - val_loss: 9.2976
Epoch 977/1000
1052/1052 [==============================] - 0s 79us/step - loss: 7.9925 - val_loss: 9.1693
Epoch 978/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.9433 - val_loss: 10.0676
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3629 - val_loss: 9.0333
Epoch 980/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2108 - val_loss: 8.8673
Epoch 981/1000
1052/1052 [======================

[array([[-1.8570180e+00, -3.9408441e+00,  7.2892344e-01,  3.1818991e+00,
          1.7379852e-01],
        [ 4.8498923e-01,  2.2124915e-01,  1.3970534e+00, -9.2249840e-02,
          1.8553475e-01],
        [ 4.1066033e-01, -7.3397189e-01,  1.5632068e+00,  1.2909333e-03,
          2.7794605e-01],
        [ 4.8958164e-02,  1.3235503e-01, -2.6381847e-01, -2.8422618e-02,
         -1.2957168e-01],
        [-3.0976489e-01, -3.3997875e-01, -3.2791993e-01,  2.3502815e+00,
          1.5243411e-01]], dtype=float32),
 array([-1.2804192 , -4.8387356 ,  0.67493284,  4.6894226 , -1.4223769 ],
       dtype=float32),
 array([[ 0.04466793, -1.1157516 , -0.80700076,  0.850554  , -0.1578632 ,
          1.0087914 , -0.7737305 , -0.16204798, -0.20315029, -0.7507175 ],
        [-1.8287063 ,  1.5374163 ,  2.4561803 , -2.0010529 ,  2.1122358 ,
         -2.3331237 ,  1.7706631 ,  2.0696056 ,  1.7042559 ,  2.2953892 ],
        [ 0.38807407, -0.11482041,  0.4736252 ,  0.04187075, -0.11231745,
         -0.5329389

In [22]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_adam_5th.h5')

In [23]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, sgd, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 161us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 71us/step - loss: 19.0296 - val_loss: 24.7764
Epoch 68/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.9917 - val_loss: 24.7270
Epoch 69/1000
1052/1052 [==============================] - 0s 66us/step - loss: 19.0083 - val_loss: 24.6411
Epoch 70/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.9563 - val_loss: 24.7192
Epoch 71/1000
1052/1052 [==============================] - 0s 74us/step - loss: 18.9562 - val_loss: 24.6903
Epoch 72/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.9329 - val_loss: 24.6950
Epoch 73/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.8910 - val_loss: 24.5853
Epoch 74/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.8874 - val_loss: 24.6242
Epoch 75/1000
1052/1052 [==============================] - 0s 73us/step - loss: 18.8790 - val_loss: 24.6663
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 72us/step - loss: 18.1398 - val_loss: 23.6217
Epoch 143/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.1491 - val_loss: 23.5853
Epoch 144/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.1428 - val_loss: 23.6358
Epoch 145/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.1278 - val_loss: 23.5856
Epoch 146/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.1093 - val_loss: 23.5990
Epoch 147/1000
1052/1052 [==============================] - 0s 73us/step - loss: 18.1022 - val_loss: 23.5681
Epoch 148/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.1062 - val_loss: 23.5830
Epoch 149/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.1036 - val_loss: 23.5322
Epoch 150/1000
1052/1052 [==============================] - 0s 75us/step - loss: 18.0864 - val_loss: 23.5389
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 71us/step - loss: 17.5793 - val_loss: 22.5872
Epoch 218/1000
1052/1052 [==============================] - 0s 69us/step - loss: 17.5663 - val_loss: 22.5646
Epoch 219/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.5663 - val_loss: 22.6015
Epoch 220/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.5668 - val_loss: 22.4797
Epoch 221/1000
1052/1052 [==============================] - 0s 76us/step - loss: 17.5389 - val_loss: 22.5409
Epoch 222/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.5487 - val_loss: 22.4812
Epoch 223/1000
1052/1052 [==============================] - 0s 69us/step - loss: 17.5515 - val_loss: 22.5067
Epoch 224/1000
1052/1052 [==============================] - 0s 74us/step - loss: 17.5271 - val_loss: 22.4250
Epoch 225/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.5445 - val_loss: 22.4943
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 73us/step - loss: 16.9112 - val_loss: 21.1550
Epoch 293/1000
1052/1052 [==============================] - 0s 71us/step - loss: 16.9040 - val_loss: 21.1735
Epoch 294/1000
1052/1052 [==============================] - 0s 74us/step - loss: 16.8967 - val_loss: 21.1824
Epoch 295/1000
1052/1052 [==============================] - 0s 69us/step - loss: 16.8728 - val_loss: 21.0583
Epoch 296/1000
1052/1052 [==============================] - 0s 72us/step - loss: 16.8578 - val_loss: 21.0947
Epoch 297/1000
1052/1052 [==============================] - 0s 70us/step - loss: 16.8521 - val_loss: 21.1024
Epoch 298/1000
1052/1052 [==============================] - 0s 73us/step - loss: 16.8439 - val_loss: 21.0233
Epoch 299/1000
1052/1052 [==============================] - 0s 74us/step - loss: 16.8181 - val_loss: 21.0047
Epoch 300/1000
1052/1052 [==============================] - 0s 73us/step - loss: 16.8137 - val_loss: 21.0172
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 69us/step - loss: 11.4273 - val_loss: 14.7422
Epoch 743/1000
1052/1052 [==============================] - 0s 70us/step - loss: 11.4247 - val_loss: 14.7012
Epoch 744/1000
1052/1052 [==============================] - 0s 71us/step - loss: 11.4426 - val_loss: 14.6602
Epoch 745/1000
1052/1052 [==============================] - 0s 69us/step - loss: 11.4135 - val_loss: 14.7690
Epoch 746/1000
1052/1052 [==============================] - 0s 69us/step - loss: 11.4051 - val_loss: 14.7190
Epoch 747/1000
1052/1052 [==============================] - 0s 63us/step - loss: 11.4081 - val_loss: 14.7119
Epoch 748/1000
1052/1052 [==============================] - 0s 73us/step - loss: 11.4024 - val_loss: 14.7125
Epoch 749/1000
1052/1052 [==============================] - 0s 75us/step - loss: 11.4100 - val_loss: 14.6344
Epoch 750/1000
1052/1052 [==============================] - 0s 67us/step - loss: 11.4008 - val_loss: 14.6492
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 73us/step - loss: 11.0885 - val_loss: 14.2939
Epoch 818/1000
1052/1052 [==============================] - 0s 73us/step - loss: 11.0887 - val_loss: 14.2357
Epoch 819/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.0879 - val_loss: 14.2335
Epoch 820/1000
1052/1052 [==============================] - 0s 74us/step - loss: 11.0898 - val_loss: 14.2891
Epoch 821/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.0803 - val_loss: 14.2548
Epoch 822/1000
1052/1052 [==============================] - 0s 71us/step - loss: 11.0821 - val_loss: 14.2173
Epoch 823/1000
1052/1052 [==============================] - 0s 73us/step - loss: 11.0839 - val_loss: 14.2155
Epoch 824/1000
1052/1052 [==============================] - 0s 69us/step - loss: 11.0621 - val_loss: 14.2658
Epoch 825/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.0695 - val_loss: 14.2456
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 74us/step - loss: 10.8599 - val_loss: 13.9288
Epoch 893/1000
1052/1052 [==============================] - 0s 74us/step - loss: 10.8552 - val_loss: 13.9472
Epoch 894/1000
1052/1052 [==============================] - 0s 73us/step - loss: 10.8625 - val_loss: 13.9582
Epoch 895/1000
1052/1052 [==============================] - 0s 67us/step - loss: 10.8396 - val_loss: 13.8982
Epoch 896/1000
1052/1052 [==============================] - 0s 74us/step - loss: 10.8361 - val_loss: 13.9240
Epoch 897/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.8431 - val_loss: 13.8874
Epoch 898/1000
1052/1052 [==============================] - 0s 71us/step - loss: 10.8321 - val_loss: 13.9246
Epoch 899/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.8285 - val_loss: 13.9174
Epoch 900/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.8258 - val_loss: 13.9165
Epoch 901/1000
1052/1052 [========

Epoch 967/1000
1052/1052 [==============================] - 0s 71us/step - loss: 10.6710 - val_loss: 13.6886
Epoch 968/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.6644 - val_loss: 13.6579
Epoch 969/1000
1052/1052 [==============================] - 0s 71us/step - loss: 10.6751 - val_loss: 13.6657
Epoch 970/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.6593 - val_loss: 13.7079
Epoch 971/1000
1052/1052 [==============================] - 0s 73us/step - loss: 10.6633 - val_loss: 13.6743
Epoch 972/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.6546 - val_loss: 13.6802
Epoch 973/1000
1052/1052 [==============================] - 0s 69us/step - loss: 10.6550 - val_loss: 13.6631
Epoch 974/1000
1052/1052 [==============================] - 0s 72us/step - loss: 10.6529 - val_loss: 13.6373
Epoch 975/1000
1052/1052 [==============================] - 0s 63us/step - loss: 10.6505 - val_loss: 13.6669
Epoch 976/1000
1052

[array([[-0.735459  ,  1.512964  , -0.9534252 ,  0.2851437 ,  0.21476173],
        [-0.3644975 ,  0.17933501,  0.08643351,  0.17940907,  0.37122297],
        [-0.25589183,  0.3030813 , -0.65861595,  0.25401184,  1.0806919 ],
        [-0.00475568, -0.12505244,  0.06604748, -0.23160335, -0.06284132],
        [-0.33606777,  0.2998244 , -0.38623354,  0.24782605,  0.47182074]],
       dtype=float32),
 array([-1.1685587 ,  0.18270569, -2.1370645 , -2.0967648 ,  0.98221433],
       dtype=float32),
 array([[ 0.34434235,  0.9961434 , -0.909173  , -0.10154836, -0.11217956,
          0.68737555, -0.836845  , -0.8146233 ,  0.39974737,  0.19688311],
        [ 0.00518829,  0.35857517, -0.20548156,  0.6715773 ,  0.02417268,
          0.20903912, -0.8605323 , -0.330376  ,  0.7886449 , -0.07456108],
        [-0.42959216,  1.7828122 , -1.6797296 ,  0.87457407,  0.1795188 ,
         -0.00354172, -1.9708226 , -0.8495077 ,  1.8167931 , -0.83034974],
        [ 0.34314525,  1.4293534 , -1.7443739 ,  1.870866

In [24]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sgd_5th.h5')

In [25]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, RMSprop, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 181us/step - l

1052/1052 [==============================] - 0s 91us/step - loss: 23.2837 - val_loss: 26.9280
Epoch 68/1000
1052/1052 [==============================] - 0s 86us/step - loss: 23.8203 - val_loss: 26.1273
Epoch 69/1000
1052/1052 [==============================] - 0s 83us/step - loss: 23.5269 - val_loss: 27.2120
Epoch 70/1000
1052/1052 [==============================] - 0s 84us/step - loss: 23.4382 - val_loss: 26.0024
Epoch 71/1000
1052/1052 [==============================] - 0s 76us/step - loss: 23.2824 - val_loss: 25.6429
Epoch 72/1000
1052/1052 [==============================] - 0s 76us/step - loss: 23.7291 - val_loss: 26.0981
Epoch 73/1000
1052/1052 [==============================] - 0s 82us/step - loss: 23.8984 - val_loss: 26.0466
Epoch 74/1000
1052/1052 [==============================] - 0s 78us/step - loss: 22.9897 - val_loss: 41.7830
Epoch 75/1000
1052/1052 [==============================] - 0s 78us/step - loss: 23.0804 - val_loss: 25.5889
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 79us/step - loss: 17.5765 - val_loss: 19.7532
Epoch 143/1000
1052/1052 [==============================] - 0s 77us/step - loss: 17.2240 - val_loss: 21.0101
Epoch 144/1000
1052/1052 [==============================] - 0s 77us/step - loss: 17.3682 - val_loss: 19.6614
Epoch 145/1000
1052/1052 [==============================] - 0s 82us/step - loss: 17.6485 - val_loss: 19.9962
Epoch 146/1000
1052/1052 [==============================] - 0s 81us/step - loss: 18.0694 - val_loss: 18.0043
Epoch 147/1000
1052/1052 [==============================] - 0s 81us/step - loss: 17.2117 - val_loss: 18.1287
Epoch 148/1000
1052/1052 [==============================] - 0s 80us/step - loss: 17.0644 - val_loss: 19.2973
Epoch 149/1000
1052/1052 [==============================] - 0s 81us/step - loss: 17.2526 - val_loss: 18.8040
Epoch 150/1000
1052/1052 [==============================] - 0s 79us/step - loss: 17.2139 - val_loss: 17.5899
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 82us/step - loss: 13.6676 - val_loss: 19.1284
Epoch 218/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.9920 - val_loss: 15.5617
Epoch 219/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.7925 - val_loss: 17.6808
Epoch 220/1000
1052/1052 [==============================] - 0s 81us/step - loss: 13.4293 - val_loss: 21.1509
Epoch 221/1000
1052/1052 [==============================] - 0s 80us/step - loss: 14.0736 - val_loss: 16.0806
Epoch 222/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.6844 - val_loss: 15.1790
Epoch 223/1000
1052/1052 [==============================] - 0s 78us/step - loss: 13.3375 - val_loss: 22.9215
Epoch 224/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.3549 - val_loss: 17.5559
Epoch 225/1000
1052/1052 [==============================] - 0s 78us/step - loss: 13.3516 - val_loss: 15.6415
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 80us/step - loss: 10.6709 - val_loss: 15.4958
Epoch 293/1000
1052/1052 [==============================] - 0s 79us/step - loss: 10.2376 - val_loss: 18.3011
Epoch 294/1000
1052/1052 [==============================] - 0s 83us/step - loss: 10.7756 - val_loss: 17.8483
Epoch 295/1000
1052/1052 [==============================] - 0s 76us/step - loss: 10.6776 - val_loss: 14.4102
Epoch 296/1000
1052/1052 [==============================] - 0s 79us/step - loss: 10.3491 - val_loss: 12.1936
Epoch 297/1000
1052/1052 [==============================] - 0s 82us/step - loss: 10.6532 - val_loss: 13.9429
Epoch 298/1000
1052/1052 [==============================] - 0s 82us/step - loss: 10.2016 - val_loss: 17.0758
Epoch 299/1000
1052/1052 [==============================] - 0s 80us/step - loss: 10.3953 - val_loss: 12.4461
Epoch 300/1000
1052/1052 [==============================] - 0s 82us/step - loss: 10.4608 - val_loss: 11.5160
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 81us/step - loss: 8.9132 - val_loss: 11.2261
Epoch 748/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2388 - val_loss: 9.5768
Epoch 749/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.9868 - val_loss: 10.6534
Epoch 750/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.5473 - val_loss: 9.7574
Epoch 751/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.1815 - val_loss: 9.9993
Epoch 752/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2673 - val_loss: 11.7348
Epoch 753/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.6738 - val_loss: 9.8735
Epoch 754/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.2969 - val_loss: 9.4000
Epoch 755/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.4375 - val_loss: 9.8320
Epoch 756/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 91us/step - loss: 9.4282 - val_loss: 10.2692
Epoch 824/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.0830 - val_loss: 9.5146
Epoch 825/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.2266 - val_loss: 9.9447
Epoch 826/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.3463 - val_loss: 9.4704
Epoch 827/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2161 - val_loss: 11.4866
Epoch 828/1000
1052/1052 [==============================] - 0s 74us/step - loss: 9.3159 - val_loss: 12.2309
Epoch 829/1000
1052/1052 [==============================] - 0s 76us/step - loss: 9.1965 - val_loss: 9.6396
Epoch 830/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.2065 - val_loss: 10.9600
Epoch 831/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.2811 - val_loss: 13.3219
Epoch 832/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 82us/step - loss: 8.9285 - val_loss: 15.3696
Epoch 900/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.5300 - val_loss: 9.3012
Epoch 901/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.1938 - val_loss: 11.8736
Epoch 902/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.8077 - val_loss: 9.5524
Epoch 903/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.8757 - val_loss: 9.2441
Epoch 904/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.3147 - val_loss: 10.5925
Epoch 905/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.6648 - val_loss: 11.6987
Epoch 906/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.7497 - val_loss: 10.2506
Epoch 907/1000
1052/1052 [==============================] - 0s 73us/step - loss: 9.3784 - val_loss: 10.1400
Epoch 908/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 78us/step - loss: 9.1630 - val_loss: 13.5370
Epoch 976/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8264 - val_loss: 9.8346
Epoch 977/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.3834 - val_loss: 9.8731
Epoch 978/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.8812 - val_loss: 11.5120
Epoch 979/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2158 - val_loss: 11.0674
Epoch 980/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.9255 - val_loss: 10.5970
Epoch 981/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1957 - val_loss: 12.3923
Epoch 982/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.3446 - val_loss: 10.5341
Epoch 983/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.1901 - val_loss: 10.2208
Epoch 984/1000
1052/1052 [===================

[array([[-3.040715  ,  1.9107448 , -0.50606513,  0.23552966,  4.00791   ],
        [ 0.0605912 , -0.42037275, -0.95976573,  0.25010136, -0.22545198],
        [-0.03241482, -0.3982603 , -1.2158897 ,  0.3858007 ,  0.7284332 ],
        [ 0.06817721, -0.04882022,  0.21853457, -0.17706281, -0.15395164],
        [-2.4575658 ,  0.26456547,  0.00555125,  0.22875123,  0.39058363]],
       dtype=float32),
 array([-4.5629473,  1.2408018, -0.643358 , -1.8417199,  4.92513  ],
       dtype=float32),
 array([[-0.8038125 , -1.6059295 ,  1.727186  , -2.1359577 , -1.252713  ,
          0.88312626, -2.0819345 ,  1.682701  ,  1.7572473 , -1.9208413 ],
        [-1.128511  , -0.18315767,  0.11853484, -0.8263691 ,  0.14108095,
          0.6052395 , -0.6848179 ,  0.93781143,  1.0299915 , -0.12960964],
        [ 0.9671655 ,  0.62195504, -0.46195695,  0.5778812 ,  0.7990196 ,
          0.09482208, -0.15707502, -0.2578131 , -0.53039825,  0.35888937],
        [-0.8796793 , -2.0494566 ,  1.3003944 , -1.408047  , -

In [26]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_rmsprop_5th.h5')

In [27]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 611us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 105us/step - loss: 6.9700 - val_loss: 8.4870
Epoch 70/1000
274/274 [==============================] - 0s 109us/step - loss: 6.9200 - val_loss: 8.4859
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8057 - val_loss: 8.6894
Epoch 72/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9418 - val_loss: 8.6097
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8893 - val_loss: 8.9353
Epoch 74/1000
274/274 [==============================] - 0s 105us/step - loss: 6.8368 - val_loss: 8.6759
Epoch 75/1000
274/274 [==============================] - 0s 106us/step - loss: 6.8823 - val_loss: 8.6250
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8130 - val_loss: 8.6621
Epoch 77/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7602 - val_loss: 8.7106
Epoch 78/1000
274/274 [==============================] -

Epoch 147/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9199 - val_loss: 8.7661
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9834 - val_loss: 8.7115
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0614 - val_loss: 9.1092
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1990 - val_loss: 9.2231
Epoch 151/1000
274/274 [==============================] - 0s 97us/step - loss: 6.0241 - val_loss: 8.7052
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0688 - val_loss: 8.8461
Epoch 153/1000
274/274 [==============================] - 0s 103us/step - loss: 5.9908 - val_loss: 8.8117
Epoch 154/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9331 - val_loss: 8.9184
Epoch 155/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0228 - val_loss: 8.6467
Epoch 156/1000
274/274 [=========================

274/274 [==============================] - 0s 102us/step - loss: 5.6988 - val_loss: 8.9727
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6714 - val_loss: 8.9155
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8483 - val_loss: 8.9942
Epoch 227/1000
274/274 [==============================] - 0s 99us/step - loss: 5.6484 - val_loss: 8.5680
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 6.4001 - val_loss: 9.3968
Epoch 229/1000
274/274 [==============================] - 0s 113us/step - loss: 6.0417 - val_loss: 8.6552
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7188 - val_loss: 8.7940
Epoch 231/1000
274/274 [==============================] - 0s 105us/step - loss: 5.9138 - val_loss: 8.8228
Epoch 232/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7907 - val_loss: 8.7065
Epoch 233/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 102us/step - loss: 5.5465 - val_loss: 8.5843
Epoch 302/1000
274/274 [==============================] - 0s 109us/step - loss: 5.5661 - val_loss: 8.7717
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6371 - val_loss: 8.6593
Epoch 304/1000
274/274 [==============================] - 0s 105us/step - loss: 5.6057 - val_loss: 8.5379
Epoch 305/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6924 - val_loss: 8.8739
Epoch 306/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8109 - val_loss: 8.8020
Epoch 307/1000
274/274 [==============================] - 0s 110us/step - loss: 5.6286 - val_loss: 8.5572
Epoch 308/1000
274/274 [==============================] - 0s 109us/step - loss: 5.9409 - val_loss: 8.7118
Epoch 309/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8959 - val_loss: 8.9334
Epoch 310/1000
274/274 [==============================] - 0s 97u

...

274/274 [==============================] - 0s 102us/step - loss: 5.5304 - val_loss: 8.5584
Epoch 764/1000
274/274 [==============================] - 0s 99us/step - loss: 5.4648 - val_loss: 8.7154
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6310 - val_loss: 8.8627
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9228 - val_loss: 8.3333
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6184 - val_loss: 8.7071
Epoch 768/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3535 - val_loss: 8.4159
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5032 - val_loss: 8.5942
Epoch 770/1000
274/274 [==============================] - 0s 101us/step - loss: 5.3999 - val_loss: 8.3709
Epoch 771/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4646 - val_loss: 8.3579
Epoch 772/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 102us/step - loss: 5.4258 - val_loss: 8.4939
Epoch 841/1000
274/274 [==============================] - 0s 100us/step - loss: 5.3539 - val_loss: 8.0538
Epoch 842/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4543 - val_loss: 8.6275
Epoch 843/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4622 - val_loss: 8.3375
Epoch 844/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7042 - val_loss: 8.3610
Epoch 845/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6065 - val_loss: 8.6498
Epoch 846/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5018 - val_loss: 8.4010
Epoch 847/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4772 - val_loss: 8.0702
Epoch 848/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4774 - val_loss: 8.5247
Epoch 849/1000
274/274 [==============================] - 0s 109

274/274 [==============================] - 0s 102us/step - loss: 5.5369 - val_loss: 8.4639
Epoch 918/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3165 - val_loss: 8.3074
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3848 - val_loss: 8.5620
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4246 - val_loss: 8.1285
Epoch 921/1000
274/274 [==============================] - 0s 97us/step - loss: 5.4652 - val_loss: 8.7795
Epoch 922/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3692 - val_loss: 8.1688
Epoch 923/1000
274/274 [==============================] - 0s 103us/step - loss: 5.5296 - val_loss: 8.7704
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3943 - val_loss: 8.3255
Epoch 925/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5567 - val_loss: 8.3373
Epoch 926/1000
274/274 [==============================] - 0s 95us

274/274 [==============================] - 0s 112us/step - loss: 5.5170 - val_loss: 8.7294
Epoch 995/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3798 - val_loss: 8.1932
Epoch 996/1000
274/274 [==============================] - 0s 101us/step - loss: 5.5304 - val_loss: 8.5574
Epoch 997/1000
274/274 [==============================] - 0s 100us/step - loss: 6.0779 - val_loss: 8.3945
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6235 - val_loss: 9.0790
Epoch 999/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4484 - val_loss: 8.2248
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4830 - val_loss: 8.5214
7.348049567917646


[array([[ 1.3014474e+00,  1.4394356e-01, -3.5244944e+00, -2.7126881e-01,
          1.2644758e+00],
        [ 6.7074114e-01,  4.1770625e+00,  1.4702228e+00, -6.0467738e-01,
         -3.2465920e-01],
        [-8.6659527e-01,  1.2346485e+00,  8.2131602e-02, -4.4805464e-01,
          3.5124767e-01],
        [-5.3141493e-01, -2.4980602e+00, -7.2767818e-01,  1.2974288e+00,
         -7.4070722e-01],
        [-4.9252494e-04,  1.1372557e+00,  3.6718842e-01, -3.8214630e-01,
         -3.5903341e-01],
        [-4.2035007e-01,  1.1405562e+00, -4.4138003e-02, -1.2157362e+00,
          1.1794764e+00],
        [-3.7090376e-01, -1.8450269e+00, -1.9506000e-01, -3.5734558e-01,
          1.1783608e+00]], dtype=float32),
 array([ 0.96984047, -1.3914226 ,  0.11576169,  1.3145385 ,  2.0928488 ],
       dtype=float32),
 array([[ 0.6160322 ,  0.9530243 ,  0.17930102, -0.9975823 ,  1.1007676 ,
         -0.7456692 , -0.3304568 ,  0.19905122, -1.0595914 , -0.10623842],
        [-0.7635004 , -0.9496925 ,  0.129199

In [28]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_adam_5th.h5')

In [29]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, sgd, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 533us/step - loss:

274/274 [==============================] - 0s 91us/step - loss: 457.4979 - val_loss: 487.6058
Epoch 68/1000
274/274 [==============================] - 0s 87us/step - loss: 454.1479 - val_loss: 484.0637
Epoch 69/1000
274/274 [==============================] - 0s 91us/step - loss: 450.8214 - val_loss: 480.3980
Epoch 70/1000
274/274 [==============================] - 0s 80us/step - loss: 447.2740 - val_loss: 476.7686
Epoch 71/1000
274/274 [==============================] - 0s 84us/step - loss: 443.8621 - val_loss: 472.9962
Epoch 72/1000
274/274 [==============================] - 0s 87us/step - loss: 440.2169 - val_loss: 469.2854
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 436.6345 - val_loss: 465.5241
Epoch 74/1000
274/274 [==============================] - 0s 91us/step - loss: 433.0244 - val_loss: 461.6418
Epoch 75/1000
274/274 [==============================] - 0s 91us/step - loss: 429.2994 - val_loss: 457.7108
Epoch 76/1000
274/274 [===================

274/274 [==============================] - 0s 91us/step - loss: 104.5993 - val_loss: 113.2502
Epoch 143/1000
274/274 [==============================] - 0s 84us/step - loss: 100.8993 - val_loss: 109.4244
Epoch 144/1000
274/274 [==============================] - 0s 80us/step - loss: 97.4577 - val_loss: 105.5694
Epoch 145/1000
274/274 [==============================] - 0s 80us/step - loss: 93.9130 - val_loss: 101.9491
Epoch 146/1000
274/274 [==============================] - 0s 84us/step - loss: 90.6813 - val_loss: 98.3487
Epoch 147/1000
274/274 [==============================] - 0s 91us/step - loss: 87.4277 - val_loss: 94.9096
Epoch 148/1000
274/274 [==============================] - 0s 87us/step - loss: 84.2928 - val_loss: 91.5864
Epoch 149/1000
274/274 [==============================] - 0s 80us/step - loss: 81.2854 - val_loss: 88.3476
Epoch 150/1000
274/274 [==============================] - 0s 80us/step - loss: 78.3644 - val_loss: 85.2125
Epoch 151/1000
274/274 [======================

274/274 [==============================] - 0s 73us/step - loss: 18.9332 - val_loss: 19.8732
Epoch 219/1000
274/274 [==============================] - 0s 80us/step - loss: 18.8622 - val_loss: 19.7944
Epoch 220/1000
274/274 [==============================] - 0s 76us/step - loss: 18.8000 - val_loss: 19.7083
Epoch 221/1000
274/274 [==============================] - 0s 73us/step - loss: 18.7325 - val_loss: 19.6279
Epoch 222/1000
274/274 [==============================] - 0s 73us/step - loss: 18.6778 - val_loss: 19.5512
Epoch 223/1000
274/274 [==============================] - 0s 76us/step - loss: 18.6171 - val_loss: 19.4855
Epoch 224/1000
274/274 [==============================] - 0s 76us/step - loss: 18.5680 - val_loss: 19.4210
Epoch 225/1000
274/274 [==============================] - 0s 80us/step - loss: 18.5141 - val_loss: 19.3575
Epoch 226/1000
274/274 [==============================] - 0s 80us/step - loss: 18.4666 - val_loss: 19.2940
Epoch 227/1000
274/274 [============================

274/274 [==============================] - 0s 80us/step - loss: 17.3684 - val_loss: 18.0346
Epoch 295/1000
274/274 [==============================] - 0s 84us/step - loss: 17.3604 - val_loss: 18.0305
Epoch 296/1000
274/274 [==============================] - 0s 77us/step - loss: 17.3510 - val_loss: 18.0229
Epoch 297/1000
274/274 [==============================] - 0s 80us/step - loss: 17.3448 - val_loss: 18.0183
Epoch 298/1000
274/274 [==============================] - 0s 84us/step - loss: 17.3347 - val_loss: 18.0098
Epoch 299/1000
274/274 [==============================] - 0s 87us/step - loss: 17.3266 - val_loss: 18.0010
Epoch 300/1000
274/274 [==============================] - 0s 84us/step - loss: 17.3189 - val_loss: 17.9946
Epoch 301/1000
274/274 [==============================] - 0s 87us/step - loss: 17.3109 - val_loss: 17.9846
Epoch 302/1000
274/274 [==============================] - 0s 76us/step - loss: 17.3040 - val_loss: 17.9764
Epoch 303/1000
274/274 [============================

...

274/274 [==============================] - 0s 95us/step - loss: 14.3476 - val_loss: 15.4943
Epoch 751/1000
274/274 [==============================] - 0s 91us/step - loss: 14.3415 - val_loss: 15.4873
Epoch 752/1000
274/274 [==============================] - 0s 91us/step - loss: 14.3351 - val_loss: 15.4825
Epoch 753/1000
274/274 [==============================] - 0s 84us/step - loss: 14.3294 - val_loss: 15.4774
Epoch 754/1000
274/274 [==============================] - 0s 84us/step - loss: 14.3232 - val_loss: 15.4710
Epoch 755/1000
274/274 [==============================] - 0s 84us/step - loss: 14.3186 - val_loss: 15.4666
Epoch 756/1000
274/274 [==============================] - 0s 95us/step - loss: 14.3103 - val_loss: 15.4602
Epoch 757/1000
274/274 [==============================] - 0s 102us/step - loss: 14.3045 - val_loss: 15.4540
Epoch 758/1000
274/274 [==============================] - 0s 109us/step - loss: 14.2987 - val_loss: 15.4473
Epoch 759/1000
274/274 [==========================

274/274 [==============================] - 0s 87us/step - loss: 13.8765 - val_loss: 15.0317
Epoch 827/1000
274/274 [==============================] - 0s 73us/step - loss: 13.8717 - val_loss: 15.0257
Epoch 828/1000
274/274 [==============================] - 0s 87us/step - loss: 13.8647 - val_loss: 15.0176
Epoch 829/1000
274/274 [==============================] - 0s 91us/step - loss: 13.8595 - val_loss: 15.0114
Epoch 830/1000
274/274 [==============================] - 0s 87us/step - loss: 13.8521 - val_loss: 15.0073
Epoch 831/1000
274/274 [==============================] - 0s 116us/step - loss: 13.8463 - val_loss: 15.0009
Epoch 832/1000
274/274 [==============================] - 0s 102us/step - loss: 13.8422 - val_loss: 14.9956
Epoch 833/1000
274/274 [==============================] - 0s 73us/step - loss: 13.8337 - val_loss: 14.9901
Epoch 834/1000
274/274 [==============================] - 0s 80us/step - loss: 13.8280 - val_loss: 14.9843
Epoch 835/1000
274/274 [==========================

274/274 [==============================] - 0s 84us/step - loss: 13.4110 - val_loss: 14.5914
Epoch 903/1000
274/274 [==============================] - 0s 73us/step - loss: 13.4055 - val_loss: 14.5845
Epoch 904/1000
274/274 [==============================] - 0s 76us/step - loss: 13.3998 - val_loss: 14.5789
Epoch 905/1000
274/274 [==============================] - 0s 80us/step - loss: 13.3946 - val_loss: 14.5722
Epoch 906/1000
274/274 [==============================] - 0s 76us/step - loss: 13.3880 - val_loss: 14.5684
Epoch 907/1000
274/274 [==============================] - 0s 76us/step - loss: 13.3813 - val_loss: 14.5625
Epoch 908/1000
274/274 [==============================] - 0s 73us/step - loss: 13.3742 - val_loss: 14.5571
Epoch 909/1000
274/274 [==============================] - 0s 73us/step - loss: 13.3689 - val_loss: 14.5503
Epoch 910/1000
274/274 [==============================] - 0s 73us/step - loss: 13.3634 - val_loss: 14.5457
Epoch 911/1000
274/274 [============================

274/274 [==============================] - 0s 80us/step - loss: 12.9514 - val_loss: 14.1276
Epoch 979/1000
274/274 [==============================] - 0s 80us/step - loss: 12.9451 - val_loss: 14.1232
Epoch 980/1000
274/274 [==============================] - 0s 73us/step - loss: 12.9390 - val_loss: 14.1193
Epoch 981/1000
274/274 [==============================] - 0s 73us/step - loss: 12.9328 - val_loss: 14.1131
Epoch 982/1000
274/274 [==============================] - 0s 80us/step - loss: 12.9272 - val_loss: 14.1064
Epoch 983/1000
274/274 [==============================] - 0s 76us/step - loss: 12.9222 - val_loss: 14.0970
Epoch 984/1000
274/274 [==============================] - 0s 76us/step - loss: 12.9146 - val_loss: 14.0919
Epoch 985/1000
274/274 [==============================] - 0s 76us/step - loss: 12.9089 - val_loss: 14.0878
Epoch 986/1000
274/274 [==============================] - 0s 82us/step - loss: 12.9032 - val_loss: 14.0829
Epoch 987/1000
274/274 [============================

[array([[-0.32678187,  0.5053104 , -0.33613378,  0.4672877 ,  0.44165695],
        [ 0.27152878, -0.4408467 , -0.12857896,  0.5828751 ,  0.18301068],
        [ 0.08714361, -0.37265682,  0.20364846,  0.6437265 , -0.3779322 ],
        [ 0.40987688,  0.5625582 ,  0.25918117, -0.16702637, -0.29402474],
        [ 0.06422357, -0.22536229,  0.21474718,  0.05188863, -0.4430298 ],
        [-0.1012086 ,  0.04594173, -0.4449722 , -0.38039973, -0.41222817],
        [ 0.26049674,  0.03159538, -0.15178236, -0.38217115, -0.69344676]],
       dtype=float32),
 array([-0.89706916, -1.2289523 , -0.65544826,  0.27893022, -0.03108319],
       dtype=float32),
 array([[-0.7750688 ,  0.909867  ,  0.13823867,  0.57529014,  0.7929965 ,
          0.4392865 , -0.2287896 ,  0.7216724 , -0.02624827, -0.28536707],
        [-0.34852847,  1.1567498 ,  0.5897626 , -0.11477017,  0.20439874,
          0.7537237 ,  1.1456362 ,  0.8554879 ,  0.81595606, -0.02504966],
        [-0.8791441 ,  0.9251317 ,  0.34827232,  0.03757

In [30]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sgd_5th.h5')

In [31]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, RMSprop, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 618us/step - loss:

274/274 [==============================] - 0s 106us/step - loss: 8.4457 - val_loss: 11.3456
Epoch 69/1000
274/274 [==============================] - 0s 95us/step - loss: 8.4524 - val_loss: 12.2241
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 8.4224 - val_loss: 13.6073
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 8.5142 - val_loss: 12.4404
Epoch 72/1000
274/274 [==============================] - 0s 94us/step - loss: 7.9687 - val_loss: 11.8257
Epoch 73/1000
274/274 [==============================] - 0s 92us/step - loss: 8.1213 - val_loss: 12.4995
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 8.3541 - val_loss: 11.8171
Epoch 75/1000
274/274 [==============================] - 0s 91us/step - loss: 8.5062 - val_loss: 11.2852
Epoch 76/1000
274/274 [==============================] - 0s 84us/step - loss: 8.9244 - val_loss: 12.1396
Epoch 77/1000
274/274 [==============================] - 0s 109us/

274/274 [==============================] - 0s 84us/step - loss: 8.4903 - val_loss: 13.1946
Epoch 146/1000
274/274 [==============================] - 0s 80us/step - loss: 7.9441 - val_loss: 11.0655
Epoch 147/1000
274/274 [==============================] - 0s 87us/step - loss: 8.0426 - val_loss: 11.1108
Epoch 148/1000
274/274 [==============================] - 0s 86us/step - loss: 7.7876 - val_loss: 15.7777
Epoch 149/1000
274/274 [==============================] - 0s 84us/step - loss: 8.9510 - val_loss: 11.1252
Epoch 150/1000
274/274 [==============================] - 0s 87us/step - loss: 8.0200 - val_loss: 11.6862
Epoch 151/1000
274/274 [==============================] - 0s 91us/step - loss: 7.8095 - val_loss: 13.1486
Epoch 152/1000
274/274 [==============================] - 0s 84us/step - loss: 8.0116 - val_loss: 14.2529
Epoch 153/1000
274/274 [==============================] - 0s 80us/step - loss: 8.5153 - val_loss: 11.6971
Epoch 154/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 87us/step - loss: 7.6653 - val_loss: 11.0906
Epoch 223/1000
274/274 [==============================] - 0s 91us/step - loss: 8.0204 - val_loss: 11.1803
Epoch 224/1000
274/274 [==============================] - 0s 104us/step - loss: 7.8151 - val_loss: 15.1456
Epoch 225/1000
274/274 [==============================] - 0s 93us/step - loss: 8.2842 - val_loss: 13.1945
Epoch 226/1000
274/274 [==============================] - 0s 97us/step - loss: 8.3573 - val_loss: 11.8700
Epoch 227/1000
274/274 [==============================] - 0s 91us/step - loss: 8.2255 - val_loss: 13.3430
Epoch 228/1000
274/274 [==============================] - 0s 87us/step - loss: 7.7356 - val_loss: 11.2817
Epoch 229/1000
274/274 [==============================] - 0s 84us/step - loss: 7.7762 - val_loss: 11.3132
Epoch 230/1000
274/274 [==============================] - 0s 84us/step - loss: 7.7018 - val_loss: 12.3092
Epoch 231/1000
274/274 [==============================] - 0s

274/274 [==============================] - 0s 84us/step - loss: 7.1222 - val_loss: 14.6297
Epoch 300/1000
274/274 [==============================] - 0s 84us/step - loss: 7.8173 - val_loss: 11.6248
Epoch 301/1000
274/274 [==============================] - 0s 87us/step - loss: 7.9538 - val_loss: 10.8751
Epoch 302/1000
274/274 [==============================] - 0s 84us/step - loss: 7.5609 - val_loss: 10.8710
Epoch 303/1000
274/274 [==============================] - 0s 87us/step - loss: 7.1703 - val_loss: 11.5938
Epoch 304/1000
274/274 [==============================] - 0s 84us/step - loss: 7.6749 - val_loss: 10.0077
Epoch 305/1000
274/274 [==============================] - 0s 84us/step - loss: 7.5193 - val_loss: 10.6134
Epoch 306/1000
274/274 [==============================] - 0s 80us/step - loss: 7.6676 - val_loss: 11.0471
Epoch 307/1000
274/274 [==============================] - 0s 80us/step - loss: 7.5513 - val_loss: 9.9217
Epoch 308/1000
274/274 [==============================] - 0s 1

...

Epoch 765/1000
274/274 [==============================] - 0s 84us/step - loss: 6.9519 - val_loss: 8.9136
Epoch 766/1000
274/274 [==============================] - 0s 80us/step - loss: 6.4955 - val_loss: 9.6921
Epoch 767/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9545 - val_loss: 8.5998
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5423 - val_loss: 10.2811
Epoch 769/1000
274/274 [==============================] - 0s 80us/step - loss: 7.3277 - val_loss: 9.6422
Epoch 770/1000
274/274 [==============================] - 0s 84us/step - loss: 6.6766 - val_loss: 10.4028
Epoch 771/1000
274/274 [==============================] - 0s 80us/step - loss: 6.7901 - val_loss: 9.2450
Epoch 772/1000
274/274 [==============================] - 0s 84us/step - loss: 6.5446 - val_loss: 8.8362
Epoch 773/1000
274/274 [==============================] - 0s 84us/step - loss: 6.3940 - val_loss: 9.7772
Epoch 774/1000
274/274 [=============================

Epoch 843/1000
274/274 [==============================] - 0s 91us/step - loss: 6.7794 - val_loss: 10.2342
Epoch 844/1000
274/274 [==============================] - 0s 87us/step - loss: 6.7360 - val_loss: 9.1056
Epoch 845/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6288 - val_loss: 8.4611
Epoch 846/1000
274/274 [==============================] - 0s 91us/step - loss: 7.1763 - val_loss: 10.1321
Epoch 847/1000
274/274 [==============================] - 0s 87us/step - loss: 6.4756 - val_loss: 9.0405
Epoch 848/1000
274/274 [==============================] - 0s 84us/step - loss: 6.5886 - val_loss: 8.5278
Epoch 849/1000
274/274 [==============================] - 0s 80us/step - loss: 6.7841 - val_loss: 10.0414
Epoch 850/1000
274/274 [==============================] - 0s 84us/step - loss: 6.6657 - val_loss: 8.8930
Epoch 851/1000
274/274 [==============================] - 0s 80us/step - loss: 6.7880 - val_loss: 9.1111
Epoch 852/1000
274/274 [============================

Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5140 - val_loss: 8.7493
Epoch 922/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5232 - val_loss: 9.6769
Epoch 923/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5737 - val_loss: 9.1654
Epoch 924/1000
274/274 [==============================] - 0s 82us/step - loss: 6.3546 - val_loss: 8.7843
Epoch 925/1000
274/274 [==============================] - 0s 84us/step - loss: 6.3402 - val_loss: 8.7102
Epoch 926/1000
274/274 [==============================] - 0s 85us/step - loss: 6.9579 - val_loss: 8.5108
Epoch 927/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2286 - val_loss: 8.5187
Epoch 928/1000
274/274 [==============================] - 0s 84us/step - loss: 6.9531 - val_loss: 8.5236
Epoch 929/1000
274/274 [==============================] - 0s 80us/step - loss: 6.8266 - val_loss: 11.7530
Epoch 930/1000
274/274 [==============================

Epoch 999/1000
274/274 [==============================] - 0s 84us/step - loss: 6.6310 - val_loss: 10.2097
Epoch 1000/1000
274/274 [==============================] - 0s 80us/step - loss: 7.0677 - val_loss: 9.2315
6.500162262027547


[array([[-1.0521784 , -1.4634523 ,  0.24550739, -0.28013533,  2.3670235 ],
        [-0.4546824 ,  0.7057921 , -0.28376585,  0.23984396, -0.8959627 ],
        [ 0.0064298 , -0.40686595,  0.281486  , -0.16083458, -0.2598355 ],
        [ 0.5495598 , -1.0339217 ,  0.24700904, -0.7688213 ,  0.02956627],
        [ 0.9584423 , -0.11621504, -0.22384478, -0.07986287,  0.01109351],
        [ 0.47399732,  0.14581347, -1.1508749 ,  0.30089086, -0.02158288],
        [-2.706805  ,  0.19745488,  0.07130721, -0.1323318 , -0.0571162 ]],
       dtype=float32),
 array([-3.4098234, -0.4919587,  0.898459 ,  2.5389068,  1.4642333],
       dtype=float32),
 array([[-0.42316115, -0.17429447, -0.19959188,  0.4047146 ,  0.7717366 ,
          0.16241725,  0.0995411 ,  0.96356   ,  0.8580517 ,  0.24760881],
        [-0.5710858 , -1.3214453 ,  0.7669146 , -0.34433833, -0.4714625 ,
         -1.1420454 , -0.73165995, -1.4162073 , -0.7130513 ,  1.10632   ],
        [-0.13199677, -0.07315946, -0.43981776,  0.9576466 , 

In [32]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_rmsprop_5th.h5')

In [33]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 98us/step - loss: 0.0059 - val_loss: 0.0059
Epoch 70/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0059 - val_loss: 0.0046
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0054 - val_loss: 0.0049
Epoch 72/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0055 - val_loss: 0.0046
Epoch 73/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0053 - val_loss: 0.0065
Epoch 74/1000
143/143 [==============================] - ETA: 0s - loss: 0.004 - 0s 91us/step - loss: 0.0061 - val_loss: 0.0048
Epoch 75/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0063 - val_loss: 0.0075
Epoch 76/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0060 - val_loss: 0.0044
Epoch 77/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0057 - val_loss: 0.0069
Epoch 78/1000
143/143 [==============================]

143/143 [==============================] - 0s 98us/step - loss: 0.0044 - val_loss: 0.0042
Epoch 148/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0045 - val_loss: 0.0040
Epoch 149/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0045 - val_loss: 0.0052
Epoch 150/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0049 - val_loss: 0.0040
Epoch 151/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0042 - val_loss: 0.0055
Epoch 152/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0047 - val_loss: 0.0042
Epoch 153/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0060 - val_loss: 0.0072
Epoch 154/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0038
Epoch 155/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0043 - val_loss: 0.0059
Epoch 156/1000
143/143 [==============================] - 0s 91us/

143/143 [==============================] - 0s 146us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 225/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0037 - val_loss: 0.0059
Epoch 226/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0042
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0038 - val_loss: 0.0036
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0048
Epoch 229/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0035 - val_loss: 0.0043
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0035
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0035 - val_loss: 0.0065
Epoch 232/1000
143/143 [==============================] - 0s 153us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 233/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0027 - val_loss: 0.0038
Epoch 302/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0036
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0034
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 305/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0028 - val_loss: 0.0044
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0034
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0039
Epoch 308/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0048
Epoch 309/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 310/1000
143/143 [==============================] - 0s 1

...

143/143 [==============================] - 0s 98us/step - loss: 0.0016 - val_loss: 0.0063
Epoch 765/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0020 - val_loss: 0.0046
Epoch 766/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 767/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0044
Epoch 768/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0019 - val_loss: 0.0053
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0018 - val_loss: 0.0046
Epoch 770/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0016 - val_loss: 0.0043
Epoch 771/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 772/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 773/1000
143/143 [==============================] - 0s 105us/

143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0046
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0050
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0050
Epoch 844/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0014 - val_loss: 0.0047
Epoch 845/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 846/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0047
Epoch 847/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 848/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0015 - val_loss: 0.0071
Epoch 849/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0023 - val_loss: 0.0043
Epoch 850/1000
143/143 [==============================] - 0s 98

143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 919/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 920/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0016 - val_loss: 0.0062
Epoch 921/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0047
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0016 - val_loss: 0.0064
Epoch 924/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0046
Epoch 925/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0054
Epoch 926/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 927/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 146us/step - loss: 0.0025 - val_loss: 0.0059
Epoch 996/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0030 - val_loss: 0.0046
Epoch 997/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0020 - val_loss: 0.0046
Epoch 998/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 999/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0047
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0045
0.013300913386046886


[array([[ 0.52834713,  0.83573353,  0.45333087, -1.0012491 ,  0.25445122],
        [ 0.46199694, -0.53941554,  0.2279119 ,  0.470724  ,  0.44075856],
        [ 0.18718725,  0.15935434, -0.65702283,  1.1528752 , -0.19122952],
        [ 0.26919115, -0.8904483 ,  0.36593682,  0.8754819 ,  0.40851948],
        [ 0.2359367 ,  0.23076123, -0.00981364,  0.4126433 ,  0.0516181 ],
        [ 0.7032039 ,  0.68483526,  0.44021496, -1.0094972 , -0.04320224],
        [ 1.272396  ,  0.75202674,  0.5903234 ,  0.18815467,  0.07545901],
        [-0.09521749, -0.24956042,  0.32168663,  0.12720127,  0.5466123 ],
        [ 0.5446358 ,  0.74242383, -0.278611  , -0.2065337 , -0.40705204],
        [-0.50552255, -0.22874223, -0.09178159,  0.09345325,  1.0397775 ],
        [ 0.14352739, -0.13192219, -1.1096328 ,  0.35714725, -0.7433681 ],
        [-1.0881563 , -0.6195883 , -0.7163196 , -0.17440686, -0.06130821],
        [-0.7716965 , -0.38335508, -0.2565474 ,  0.09834147,  0.02720609],
        [-0.11780426, -0.

In [34]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_adam_5th.h5')

In [35]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, sgd, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 101us/step - loss: 0.0930 - val_loss: 0.0909
Epoch 70/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0928 - val_loss: 0.0907
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0926 - val_loss: 0.0905
Epoch 72/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0924 - val_loss: 0.0902
Epoch 73/1000
143/143 [==============================] - 0s 95us/step - loss: 0.0921 - val_loss: 0.0900
Epoch 74/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0919 - val_loss: 0.0898
Epoch 75/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0917 - val_loss: 0.0896
Epoch 76/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0915 - val_loss: 0.0893
Epoch 77/1000
143/143 [==============================] - 0s 100us/step - loss: 0.0913 - val_loss: 0.0891
Epoch 78/1000
143/143 [==============================] - 0s 100us/step - l

143/143 [==============================] - 0s 98us/step - loss: 0.0788 - val_loss: 0.0759
Epoch 148/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0787 - val_loss: 0.0758
Epoch 149/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0785 - val_loss: 0.0756
Epoch 150/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0784 - val_loss: 0.0754
Epoch 151/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0782 - val_loss: 0.0753
Epoch 152/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0781 - val_loss: 0.0751
Epoch 153/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0779 - val_loss: 0.0750
Epoch 154/1000
143/143 [==============================] - 0s 80us/step - loss: 0.0778 - val_loss: 0.0748
Epoch 155/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0776 - val_loss: 0.0747
Epoch 156/1000
143/143 [==============================] - 0s 91us/step

143/143 [==============================] - 0s 89us/step - loss: 0.0689 - val_loss: 0.0652
Epoch 226/1000
143/143 [==============================] - 0s 90us/step - loss: 0.0688 - val_loss: 0.0651
Epoch 227/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0687 - val_loss: 0.0650
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0686 - val_loss: 0.0649
Epoch 229/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0685 - val_loss: 0.0648
Epoch 230/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0684 - val_loss: 0.0647
Epoch 231/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0683 - val_loss: 0.0645
Epoch 232/1000
143/143 [==============================] - 0s 92us/step - loss: 0.0682 - val_loss: 0.0644
Epoch 233/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0681 - val_loss: 0.0643
Epoch 234/1000
143/143 [==============================] - 0s 84us/ste

143/143 [==============================] - 0s 91us/step - loss: 0.0619 - val_loss: 0.0575
Epoch 304/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0618 - val_loss: 0.0574
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0617 - val_loss: 0.0573
Epoch 306/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0617 - val_loss: 0.0572
Epoch 307/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0616 - val_loss: 0.0571
Epoch 308/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0615 - val_loss: 0.0571
Epoch 309/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0614 - val_loss: 0.0570
Epoch 310/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0614 - val_loss: 0.0569
Epoch 311/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0613 - val_loss: 0.0568
Epoch 312/1000
143/143 [==============================] - 0s 84us/ste

...

143/143 [==============================] - 0s 91us/step - loss: 0.0475 - val_loss: 0.0404
Epoch 693/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0475 - val_loss: 0.0404
Epoch 694/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0475 - val_loss: 0.0404
Epoch 695/1000
143/143 [==============================] - 0s 88us/step - loss: 0.0475 - val_loss: 0.0404
Epoch 696/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0475 - val_loss: 0.0403
Epoch 697/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0475 - val_loss: 0.0403
Epoch 698/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0474 - val_loss: 0.0403
Epoch 699/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0474 - val_loss: 0.0403
Epoch 700/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0474 - val_loss: 0.0403
Epoch 701/1000
143/143 [==============================] - 0s 91us/step

Epoch 770/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0465 - val_loss: 0.0391
Epoch 771/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0465 - val_loss: 0.0390
Epoch 772/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0465 - val_loss: 0.0390
Epoch 773/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0465 - val_loss: 0.0390
Epoch 774/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0465 - val_loss: 0.0390
Epoch 775/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0465 - val_loss: 0.0390
Epoch 776/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0464 - val_loss: 0.0390
Epoch 777/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0464 - val_loss: 0.0389
Epoch 778/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0464 - val_loss: 0.0389
Epoch 779/1000
143/143 [=============================

143/143 [==============================] - 0s 98us/step - loss: 0.0457 - val_loss: 0.0380
Epoch 848/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0457 - val_loss: 0.0380
Epoch 849/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 850/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 851/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 852/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 853/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 854/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 855/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0457 - val_loss: 0.0379
Epoch 856/1000
143/143 [==============================] - 0s 91us/step

143/143 [==============================] - 0s 98us/step - loss: 0.0451 - val_loss: 0.0371
Epoch 926/1000
143/143 [==============================] - 0s 89us/step - loss: 0.0451 - val_loss: 0.0371
Epoch 927/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0451 - val_loss: 0.0371
Epoch 928/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0451 - val_loss: 0.0371
Epoch 929/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0451 - val_loss: 0.0371
Epoch 930/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0451 - val_loss: 0.0371
Epoch 931/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0451 - val_loss: 0.0370
Epoch 932/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0451 - val_loss: 0.0370
Epoch 933/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0451 - val_loss: 0.0370
Epoch 934/1000
143/143 [==============================] - 0s 91us/step

[array([[ 0.15466543,  0.33500695, -0.22603598,  0.24202079, -0.25258473],
        [-0.38218412, -0.40062088, -0.16802473, -0.3437878 , -0.42388374],
        [ 0.0499099 , -0.0906921 ,  0.3684227 ,  0.00163446,  0.1719199 ],
        [-0.09412647,  0.00139144,  0.08812755, -0.18132171, -0.16838527],
        [-0.06312627,  0.34138477, -0.22034194,  0.0208194 ,  0.19688359],
        [ 0.04664409,  0.16729963, -0.25738886, -0.09335659,  0.10639455],
        [ 0.11686406, -0.07738612, -0.31602105,  0.3241451 , -0.3652046 ],
        [ 0.38073236,  0.36754438,  0.21985823,  0.08885082,  0.07217958],
        [ 0.1605972 ,  0.33656296, -0.46458915, -0.27802265, -0.04338146],
        [-0.3377804 , -0.12520391, -0.30150592,  0.06115614,  0.21955965],
        [ 0.39431262,  0.30429143, -0.10956448, -0.04822128,  0.26329905],
        [ 0.36336455, -0.31330788, -0.43760756,  0.203722  , -0.09788213],
        [ 0.10529505, -0.02653422, -0.32440254, -0.28455693,  0.32107738],
        [-0.08387168,  0.

In [36]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sgd_5th.h5')

In [37]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, RMSprop, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 98us/step - loss: 0.0131 - val_loss: 0.0109
Epoch 70/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0094 - val_loss: 0.0101
Epoch 71/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0103 - val_loss: 0.0054
Epoch 72/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0095 - val_loss: 0.0134
Epoch 73/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0149 - val_loss: 0.0053
Epoch 74/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0086 - val_loss: 0.0119
Epoch 75/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0103 - val_loss: 0.0096
Epoch 76/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0098 - val_loss: 0.0050
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0086 - val_loss: 0.0064
Epoch 78/1000
143/143 [==============================] - 0s 102us/step - loss

143/143 [==============================] - 0s 112us/step - loss: 0.0088 - val_loss: 0.0049
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0075 - val_loss: 0.0057
Epoch 149/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0073 - val_loss: 0.0092
Epoch 150/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0097 - val_loss: 0.0054
Epoch 151/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0077 - val_loss: 0.0045
Epoch 152/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0067 - val_loss: 0.0092
Epoch 153/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0095 - val_loss: 0.0043
Epoch 154/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0075 - val_loss: 0.0065
Epoch 155/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0070 - val_loss: 0.0129
Epoch 156/1000
143/143 [==============================] - 0s 105us/s

Epoch 225/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0061 - val_loss: 0.0067
Epoch 226/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0059 - val_loss: 0.0059
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0070 - val_loss: 0.0068
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0087 - val_loss: 0.0071
Epoch 229/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0061 - val_loss: 0.0116
Epoch 230/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0070 - val_loss: 0.0045
Epoch 231/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0060 - val_loss: 0.0049
Epoch 232/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0058 - val_loss: 0.0082
Epoch 233/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0077 - val_loss: 0.0058
Epoch 234/1000
143/143 [=========================

143/143 [==============================] - 0s 105us/step - loss: 0.0074 - val_loss: 0.0047
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0082
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0059 - val_loss: 0.0054
Epoch 305/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 306/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0049 - val_loss: 0.0036
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0057 - val_loss: 0.0042
Epoch 308/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0062 - val_loss: 0.0047
Epoch 309/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0078 - val_loss: 0.0042
Epoch 310/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0053 - val_loss: 0.0061
Epoch 311/1000
143/143 [==============================] - 0s 91

...

143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0061
Epoch 765/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0041 - val_loss: 0.0029
Epoch 766/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0035 - val_loss: 0.0021
Epoch 767/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0060 - val_loss: 0.0052
Epoch 768/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0037 - val_loss: 0.0023
Epoch 769/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0040 - val_loss: 0.0026
Epoch 770/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0037 - val_loss: 0.0077
Epoch 771/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0048 - val_loss: 0.0045
Epoch 772/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0044 - val_loss: 0.0077
Epoch 773/1000
143/143 [==============================] - 0s 112us/

143/143 [==============================] - 0s 98us/step - loss: 0.0050 - val_loss: 0.0021
Epoch 842/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0041 - val_loss: 0.0046
Epoch 843/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0038 - val_loss: 0.0019
Epoch 844/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0034 - val_loss: 0.0035
Epoch 845/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0041 - val_loss: 0.0024
Epoch 846/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0051 - val_loss: 0.0023
Epoch 847/1000
143/143 [==============================] - 0s 95us/step - loss: 0.0038 - val_loss: 0.0032
Epoch 848/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0040 - val_loss: 0.0025
Epoch 849/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0036 - val_loss: 0.0031
Epoch 850/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 98us/step - loss: 0.0033 - val_loss: 0.0034
Epoch 919/1000
143/143 [==============================] - 0s 99us/step - loss: 0.0076 - val_loss: 0.0027
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0038
Epoch 921/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0037 - val_loss: 0.0042
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0021
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0020
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0042
Epoch 925/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0044
Epoch 926/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0045 - val_loss: 0.0025
Epoch 927/1000
143/143 [==============================] - 0s 10

143/143 [==============================] - 0s 109us/step - loss: 0.0044 - val_loss: 0.0027
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0075
Epoch 997/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0041 - val_loss: 0.0024
Epoch 998/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0031 - val_loss: 0.0024
Epoch 999/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0041 - val_loss: 0.0023
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0034 - val_loss: 0.0048
0.007797514088451862


[array([[ 0.9145945 , -0.12913501, -0.24642639,  0.40171334,  1.3680431 ],
        [-0.8668023 , -0.16185826,  0.18652105, -0.33462644, -1.1859984 ],
        [ 0.44055393,  0.1127468 , -0.23819147, -0.10835653, -0.8668959 ],
        [ 0.23763494, -0.19943859, -0.19944164, -0.09752297,  0.20187858],
        [-0.41105303, -0.02768229, -0.04241125,  0.37080032, -0.3242732 ],
        [-0.96965456,  0.03093935,  0.26701793,  0.54928356,  0.36871928],
        [ 0.30110186, -0.05183277, -0.06449451, -0.4593259 , -0.04613569],
        [ 0.43741667,  0.25347435,  0.0487202 , -0.61273867,  0.25911692],
        [-1.2832496 ,  0.20398283,  0.03882313,  0.22506858, -0.11413242],
        [ 0.47845155, -0.370283  ,  0.2078344 , -1.0852559 ,  0.44459882],
        [ 0.20444775, -0.1775229 , -0.23451023, -0.1834604 , -0.44442996],
        [-0.11470698, -0.190131  , -0.12929584, -0.6075552 , -0.10453427],
        [ 0.8871584 ,  0.16898948, -0.08562088, -0.675391  , -0.35313955],
        [-2.0737097 , -0.

In [38]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_rmsprop_5th.h5')

In [39]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 7ms/step - loss: 35.939

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0157 - val_loss: 0.0090
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0163 - val_loss: 0.0092
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0165 - val_loss: 0.0088
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0162 - val_loss: 0.0080
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0156 - val_loss: 0.0071
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0148 - val_loss: 0.0066
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0141 - val_loss: 0.0066
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0136 - val_loss: 0.0069
Epoch 79/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0131 - val_loss: 0.0073
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - los

32/32 [==============================] - 0s 125us/step - loss: 0.0053 - val_loss: 0.0013
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0053 - val_loss: 0.0013
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0052 - val_loss: 0.0013
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0052 - val_loss: 0.0013
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0052 - val_loss: 0.0013
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0051 - val_loss: 0.0012
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0051 - val_loss: 0.0012
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0051 - val_loss: 0.0012
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0050 - val_loss: 0.0012
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0

Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 6.8125e-04
Epoch 228/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0036 - val_loss: 6.7683e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 6.7247e-04
Epoch 230/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0035 - val_loss: 6.6822e-04
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0035 - val_loss: 6.6407e-04
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0035 - val_loss: 6.6001e-04
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 6.5604e-04
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 6.5216e-04
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0035 - val_loss: 6.4834e-04
Epoch 236/1000
32/32 [======

Epoch 303/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0028 - val_loss: 4.7526e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 4.7355e-04
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 4.7188e-04
Epoch 306/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0027 - val_loss: 4.7021e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 4.6855e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 4.6692e-04
Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 4.6530e-04
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 4.6370e-04
Epoch 311/1000
32/32 [==============================] - 0s 121us/step - loss: 0.0027 - val_loss: 4.6210e-04
Epoch 312/1000
32/32 [======

...

Epoch 759/1000
32/32 [==============================] - 0s 125us/step - loss: 9.9611e-04 - val_loss: 1.7855e-04
Epoch 760/1000
32/32 [==============================] - 0s 125us/step - loss: 9.9420e-04 - val_loss: 1.7823e-04
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9231e-04 - val_loss: 1.7793e-04
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 9.9042e-04 - val_loss: 1.7761e-04
Epoch 763/1000
32/32 [==============================] - 0s 187us/step - loss: 9.8851e-04 - val_loss: 1.7730e-04
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8662e-04 - val_loss: 1.7699e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8474e-04 - val_loss: 1.7669e-04
Epoch 766/1000
32/32 [==============================] - 0s 124us/step - loss: 9.8287e-04 - val_loss: 1.7637e-04
Epoch 767/1000
32/32 [==============================] - 0s 156us/step - loss: 9.8099e-04 - val_loss: 1.7

Epoch 833/1000
32/32 [==============================] - 0s 125us/step - loss: 8.6640e-04 - val_loss: 1.5711e-04
Epoch 834/1000
32/32 [==============================] - 0s 125us/step - loss: 8.6479e-04 - val_loss: 1.5683e-04
Epoch 835/1000
32/32 [==============================] - 0s 124us/step - loss: 8.6319e-04 - val_loss: 1.5656e-04
Epoch 836/1000
32/32 [==============================] - 0s 124us/step - loss: 8.6159e-04 - val_loss: 1.5629e-04
Epoch 837/1000
32/32 [==============================] - 0s 155us/step - loss: 8.5999e-04 - val_loss: 1.5603e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 8.5840e-04 - val_loss: 1.5576e-04
Epoch 839/1000
32/32 [==============================] - 0s 125us/step - loss: 8.5681e-04 - val_loss: 1.5549e-04
Epoch 840/1000
32/32 [==============================] - 0s 125us/step - loss: 8.5521e-04 - val_loss: 1.5523e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 8.5364e-04 - val_loss: 1.5

Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5659e-04 - val_loss: 1.3854e-04
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5522e-04 - val_loss: 1.3830e-04
Epoch 909/1000
32/32 [==============================] - 0s 124us/step - loss: 7.5386e-04 - val_loss: 1.3807e-04
Epoch 910/1000
32/32 [==============================] - 0s 124us/step - loss: 7.5249e-04 - val_loss: 1.3784e-04
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5114e-04 - val_loss: 1.3761e-04
Epoch 912/1000
32/32 [==============================] - 0s 125us/step - loss: 7.4979e-04 - val_loss: 1.3737e-04
Epoch 913/1000
32/32 [==============================] - 0s 125us/step - loss: 7.4843e-04 - val_loss: 1.3714e-04
Epoch 914/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4709e-04 - val_loss: 1.3691e-04
Epoch 915/1000
32/32 [==============================] - 0s 125us/step - loss: 7.4573e-04 - val_loss: 1.3

Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6302e-04 - val_loss: 1.2241e-04
Epoch 982/1000
32/32 [==============================] - 0s 124us/step - loss: 6.6185e-04 - val_loss: 1.2221e-04
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6069e-04 - val_loss: 1.2201e-04
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5953e-04 - val_loss: 1.2179e-04
Epoch 985/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5836e-04 - val_loss: 1.2160e-04
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5721e-04 - val_loss: 1.2140e-04
Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5605e-04 - val_loss: 1.2120e-04
Epoch 988/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5490e-04 - val_loss: 1.2100e-04
Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5375e-04 - val_loss: 1.2

[array([[ 0.09056102, -0.14724083, -0.06855701,  0.17146902, -0.3879984 ],
        [ 0.27192226,  0.921981  , -0.9026538 ,  0.77696645, -0.12618859],
        [-0.1954985 , -0.44994113, -0.87345314, -0.84742135, -1.5475674 ]],
       dtype=float32),
 array([-0.44390717,  0.5711888 ,  0.5365945 ,  0.6602212 , -0.46037558],
       dtype=float32),
 array([[-0.11531635, -0.44183537,  0.258087  ,  0.62827104, -0.00892567,
          0.20633645, -0.4556343 ,  0.44189495, -0.10829671, -0.09155259],
        [ 0.00439368,  0.10963328,  0.59258795,  0.11617761, -0.23604786,
          0.05818256,  0.2996943 , -0.3087977 ,  0.17428672, -0.22776653],
        [ 0.2885124 , -0.00334982, -0.09641812,  0.33401057,  0.6127034 ,
         -0.3943197 , -0.08687043, -0.56791085,  0.3044911 , -0.74831146],
        [ 0.5503855 , -0.03132147,  0.5849338 , -0.32801402,  0.26754233,
          0.34085748,  0.69905037,  0.06639531,  0.18797442,  0.03203891],
        [-0.6844831 , -0.7957165 ,  0.17100662,  0.2619339

In [40]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_adam_5th.h5')

In [41]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, sgd, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 36.014

Epoch 70/1000
32/32 [==============================] - 0s 94us/step - loss: 35.4459 - val_loss: 39.4803
Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4366 - val_loss: 39.4701
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4274 - val_loss: 39.4600
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4181 - val_loss: 39.4498
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4089 - val_loss: 39.4396
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 35.3996 - val_loss: 39.4294
Epoch 76/1000
32/32 [==============================] - 0s 101us/step - loss: 35.3904 - val_loss: 39.4193
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 35.3811 - val_loss: 39.4091
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 35.3719 - val_loss: 39.3989
Epoch 79/1000
32/32 [==============================] - 0

Epoch 148/1000
32/32 [==============================] - 0s 124us/step - loss: 34.7309 - val_loss: 38.6945
Epoch 149/1000
32/32 [==============================] - 0s 94us/step - loss: 34.7219 - val_loss: 38.6845
Epoch 150/1000
32/32 [==============================] - 0s 125us/step - loss: 34.7128 - val_loss: 38.6745
Epoch 151/1000
32/32 [==============================] - 0s 124us/step - loss: 34.7037 - val_loss: 38.6646
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 34.6947 - val_loss: 38.6546
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 34.6856 - val_loss: 38.6447
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 34.6765 - val_loss: 38.6347
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 34.6675 - val_loss: 38.6248
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 34.6584 - val_loss: 38.6148
Epoch 157/1000
32/32 [=========================

Epoch 226/1000
32/32 [==============================] - 0s 124us/step - loss: 34.0302 - val_loss: 37.9252
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 34.0213 - val_loss: 37.9154
Epoch 228/1000
32/32 [==============================] - 0s 125us/step - loss: 34.0124 - val_loss: 37.9057
Epoch 229/1000
32/32 [==============================] - 0s 125us/step - loss: 34.0035 - val_loss: 37.8959
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 33.9946 - val_loss: 37.8862
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 33.9857 - val_loss: 37.8764
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9768 - val_loss: 37.8667
Epoch 233/1000
32/32 [==============================] - 0s 124us/step - loss: 33.9679 - val_loss: 37.8569
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 33.9590 - val_loss: 37.8472
Epoch 235/1000
32/32 [========================

Epoch 304/1000
32/32 [==============================] - 0s 125us/step - loss: 33.3423 - val_loss: 37.1710
Epoch 305/1000
32/32 [==============================] - 0s 125us/step - loss: 33.3336 - val_loss: 37.1614
Epoch 306/1000
32/32 [==============================] - 0s 156us/step - loss: 33.3248 - val_loss: 37.1518
Epoch 307/1000
32/32 [==============================] - 0s 125us/step - loss: 33.3161 - val_loss: 37.1423
Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 33.3074 - val_loss: 37.1327
Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 33.2986 - val_loss: 37.1231
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 33.2899 - val_loss: 37.1136
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 33.2812 - val_loss: 37.1040
Epoch 312/1000
32/32 [==============================] - 0s 124us/step - loss: 33.2724 - val_loss: 37.0944
Epoch 313/1000
32/32 [========================

...

Epoch 694/1000
32/32 [==============================] - 0s 124us/step - loss: 30.0583 - val_loss: 33.5811
Epoch 695/1000
32/32 [==============================] - 0s 125us/step - loss: 30.0502 - val_loss: 33.5722
Epoch 696/1000
32/32 [==============================] - 0s 125us/step - loss: 30.0420 - val_loss: 33.5633
Epoch 697/1000
32/32 [==============================] - 0s 94us/step - loss: 30.0339 - val_loss: 33.5544
Epoch 698/1000
32/32 [==============================] - 0s 125us/step - loss: 30.0258 - val_loss: 33.5455
Epoch 699/1000
32/32 [==============================] - 0s 125us/step - loss: 30.0176 - val_loss: 33.5366
Epoch 700/1000
32/32 [==============================] - 0s 125us/step - loss: 30.0095 - val_loss: 33.5277
Epoch 701/1000
32/32 [==============================] - 0s 94us/step - loss: 30.0013 - val_loss: 33.5189
Epoch 702/1000
32/32 [==============================] - 0s 124us/step - loss: 29.9932 - val_loss: 33.5100
Epoch 703/1000
32/32 [==========================

Epoch 772/1000
32/32 [==============================] - 0s 124us/step - loss: 29.4263 - val_loss: 32.8917
Epoch 773/1000
32/32 [==============================] - 0s 155us/step - loss: 29.4182 - val_loss: 32.8829
Epoch 774/1000
32/32 [==============================] - 0s 125us/step - loss: 29.4101 - val_loss: 32.8741
Epoch 775/1000
32/32 [==============================] - 0s 125us/step - loss: 29.4021 - val_loss: 32.8654
Epoch 776/1000
32/32 [==============================] - 0s 125us/step - loss: 29.3940 - val_loss: 32.8566
Epoch 777/1000
32/32 [==============================] - 0s 125us/step - loss: 29.3860 - val_loss: 32.8478
Epoch 778/1000
32/32 [==============================] - 0s 124us/step - loss: 29.3779 - val_loss: 32.8390
Epoch 779/1000
32/32 [==============================] - 0s 93us/step - loss: 29.3699 - val_loss: 32.8302
Epoch 780/1000
32/32 [==============================] - 0s 124us/step - loss: 29.3618 - val_loss: 32.8215
Epoch 781/1000
32/32 [=========================

Epoch 850/1000
32/32 [==============================] - 0s 125us/step - loss: 28.8007 - val_loss: 32.2099
Epoch 851/1000
32/32 [==============================] - 0s 125us/step - loss: 28.7927 - val_loss: 32.2012
Epoch 852/1000
32/32 [==============================] - 0s 125us/step - loss: 28.7847 - val_loss: 32.1925
Epoch 853/1000
32/32 [==============================] - 0s 125us/step - loss: 28.7768 - val_loss: 32.1838
Epoch 854/1000
32/32 [==============================] - 0s 125us/step - loss: 28.7688 - val_loss: 32.1751
Epoch 855/1000
32/32 [==============================] - 0s 156us/step - loss: 28.7608 - val_loss: 32.1664
Epoch 856/1000
32/32 [==============================] - 0s 125us/step - loss: 28.7528 - val_loss: 32.1577
Epoch 857/1000
32/32 [==============================] - 0s 93us/step - loss: 28.7449 - val_loss: 32.1490
Epoch 858/1000
32/32 [==============================] - 0s 125us/step - loss: 28.7369 - val_loss: 32.1403
Epoch 859/1000
32/32 [=========================

Epoch 928/1000
32/32 [==============================] - 0s 124us/step - loss: 28.1811 - val_loss: 31.5347
Epoch 929/1000
32/32 [==============================] - 0s 126us/step - loss: 28.1732 - val_loss: 31.5261
Epoch 930/1000
32/32 [==============================] - 0s 140us/step - loss: 28.1652 - val_loss: 31.5175
Epoch 931/1000
32/32 [==============================] - 0s 125us/step - loss: 28.1573 - val_loss: 31.5089
Epoch 932/1000
32/32 [==============================] - 0s 156us/step - loss: 28.1494 - val_loss: 31.5003
Epoch 933/1000
32/32 [==============================] - 0s 124us/step - loss: 28.1415 - val_loss: 31.4917
Epoch 934/1000
32/32 [==============================] - 0s 124us/step - loss: 28.1336 - val_loss: 31.4831
Epoch 935/1000
32/32 [==============================] - 0s 124us/step - loss: 28.1257 - val_loss: 31.4744
Epoch 936/1000
32/32 [==============================] - 0s 124us/step - loss: 28.1178 - val_loss: 31.4658
Epoch 937/1000
32/32 [========================

[array([[ 0.6969341 ,  0.06979498,  0.28582537,  0.41822234,  0.57309145],
        [-0.83487827,  0.13838951, -0.31552014, -0.15850592,  0.01319952],
        [ 0.23510711,  0.60765356,  0.4974175 , -0.6459303 , -0.79658496]],
       dtype=float32),
 array([-0.04482594, -0.11760397,  0.04541907,  0.12816344, -0.04478743],
       dtype=float32),
 array([[-0.12922443, -0.24938425,  0.37650347, -0.34034374,  0.4925372 ,
         -0.14596096,  0.23394717, -0.51161826,  0.32250664, -0.47266975],
        [-0.4601581 , -0.20304288,  0.12898397,  0.35232562, -0.21782877,
         -0.5323065 ,  0.54631037, -0.6076844 ,  0.11552908,  0.18493894],
        [ 0.40989572,  0.51775783, -0.45437357, -0.6242968 ,  0.5159074 ,
          0.00154999,  0.58743036,  0.43919536, -0.5861541 ,  0.28069952],
        [ 0.3327614 , -0.57590026, -0.6258745 ,  0.03992485, -0.4739152 ,
         -0.13125879, -0.11803488,  0.28011855,  0.4137189 ,  0.18756658],
        [-0.4215489 ,  0.27784476, -0.01601287, -0.2986401

In [42]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sgd_5th.h5')

In [43]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, RMSprop, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 7ms/step - loss: 36.695

Epoch 71/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0857 - val_loss: 0.0590
Epoch 72/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0675 - val_loss: 0.0635
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0501 - val_loss: 0.0407
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0445 - val_loss: 0.0520
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0404 - val_loss: 0.0353
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0408 - val_loss: 0.0522
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0428 - val_loss: 0.0350
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0505 - val_loss: 0.0658
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0615 - val_loss: 0.0493
Epoch 80/1000
32/32 [==============================] - 0s 125us/step - lo

32/32 [==============================] - 0s 156us/step - loss: 0.0866 - val_loss: 0.0444
Epoch 151/1000
32/32 [==============================] - 0s 123us/step - loss: 0.0813 - val_loss: 0.0369
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0600 - val_loss: 0.0226
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0358 - val_loss: 0.0190
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0267 - val_loss: 0.0204
Epoch 155/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0235 - val_loss: 0.0247
Epoch 156/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0291 - val_loss: 0.0468
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0434 - val_loss: 0.0720
Epoch 158/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0747 - val_loss: 0.1071
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.

Epoch 229/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0347 - val_loss: 0.0246
Epoch 230/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0348 - val_loss: 0.0251
Epoch 231/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0340 - val_loss: 0.0246
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0392 - val_loss: 0.0270
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0404 - val_loss: 0.0254
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0428 - val_loss: 0.0258
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0386 - val_loss: 0.0231
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0368 - val_loss: 0.0256
Epoch 237/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0335 - val_loss: 0.0291
Epoch 238/1000
32/32 [==============================] - 0s 124us

Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0257 - val_loss: 0.0471
Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0487 - val_loss: 0.0965
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0952 - val_loss: 0.1098
Epoch 311/1000
32/32 [==============================] - 0s 126us/step - loss: 0.1069 - val_loss: 0.1013
Epoch 312/1000
32/32 [==============================] - 0s 155us/step - loss: 0.1032 - val_loss: 0.0573
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0655 - val_loss: 0.0419
Epoch 314/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0575 - val_loss: 0.0272
Epoch 315/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0420 - val_loss: 0.0210
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0351 - val_loss: 0.0161
Epoch 317/1000
32/32 [==============================] - 0s 125us

...

Epoch 703/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0529 - val_loss: 0.0395
Epoch 704/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0439 - val_loss: 0.0438
Epoch 705/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0477 - val_loss: 0.0362
Epoch 706/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0407 - val_loss: 0.0349
Epoch 707/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0392 - val_loss: 0.0248
Epoch 708/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0302 - val_loss: 0.0229
Epoch 709/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0275 - val_loss: 0.0166
Epoch 710/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0224 - val_loss: 0.0167
Epoch 711/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0217 - val_loss: 0.0134
Epoch 712/1000
32/32 [==============================] - 0s 156us

Epoch 782/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0137 - val_loss: 0.0100
Epoch 783/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0142 - val_loss: 0.0103
Epoch 784/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0150 - val_loss: 0.0141
Epoch 785/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0185 - val_loss: 0.0174
Epoch 786/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0227 - val_loss: 0.0271
Epoch 787/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0322 - val_loss: 0.0346
Epoch 788/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0411 - val_loss: 0.0504
Epoch 789/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0564 - val_loss: 0.0493
Epoch 790/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0558 - val_loss: 0.0530
Epoch 791/1000
32/32 [==============================] - 0s 156us

Epoch 861/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0372 - val_loss: 0.0226
Epoch 862/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0252 - val_loss: 0.0208
Epoch 863/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0226 - val_loss: 0.0161
Epoch 864/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0209 - val_loss: 0.0192
Epoch 865/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0260 - val_loss: 0.0178
Epoch 866/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0307 - val_loss: 0.0208
Epoch 867/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0353 - val_loss: 0.0150
Epoch 868/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0290 - val_loss: 0.0134
Epoch 869/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0229 - val_loss: 0.0086
Epoch 870/1000
32/32 [==============================] - 0s 125us

Epoch 940/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0128 - val_loss: 0.0164
Epoch 941/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0182 - val_loss: 0.0267
Epoch 942/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0293 - val_loss: 0.0530
Epoch 943/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0526 - val_loss: 0.0661
Epoch 944/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0671 - val_loss: 0.0763
Epoch 945/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0765 - val_loss: 0.0484
Epoch 946/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0542 - val_loss: 0.0389
Epoch 947/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0469 - val_loss: 0.0223
Epoch 948/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0314 - val_loss: 0.0176
Epoch 949/1000
32/32 [==============================] - 0s 125us/

[array([[ 0.51263875,  0.80896986,  0.00595985,  0.20694405,  0.5027192 ],
        [-0.22530574, -0.08534189,  0.05866665, -0.05954516,  0.20408928],
        [-0.3991134 ,  0.1381345 ,  0.18508072, -0.0802011 ,  0.3857217 ]],
       dtype=float32),
 array([ 0.28211   , -0.37899697,  0.41298273,  0.33775887,  0.32688272],
       dtype=float32),
 array([[ 0.448501  ,  0.21534765, -0.37219357, -0.10272112,  0.2606449 ,
          0.62140864, -0.6213567 ,  0.23469952,  0.42477912, -0.4227617 ],
        [-0.06083927,  0.52766275,  0.53080344,  0.27403927,  0.14151901,
         -0.83799034,  0.44434944,  0.08110259, -0.42612296,  0.37949812],
        [ 0.66655433,  0.13871539, -0.29017702,  0.24338292, -0.2667478 ,
          0.2753393 , -0.31192723, -0.7764916 ,  0.46544275, -0.6585202 ],
        [ 0.6014414 ,  0.42050758, -0.56431097, -0.50156593, -0.5535834 ,
          0.4605099 , -0.40957552, -0.67721295, -0.09823111,  0.47671756],
        [-0.02861236, -0.44147468, -0.29446372,  0.1984764

In [44]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_rmsprop_5th.h5')